Ovo je model koji služi za prepoznavanje izgleda bolida tima u Formuli 1 na osnovu date slike. Ukupno postoji 10 timova: Mercedes, McLaren, Williams, Red Bull, Racing Bulls, Kick Sauber, Ferrari, Haas, Alpine i Aston Martin. Dataset ukupno sadrži 983 slike bolida. Kao baza ovog modela koristi se pretrenirani model ResNet50 koji je pogodan za detekciju slika. U prvom koraku koristi se tzv. ekstrakcija osobina (feature extraction), gde se izlazi iz baznog modela koriste kao ulazi za dodatne, sopstvene slojeve modela. Nakon toga otključavamo bazu modela radi tzv. fine-tuninga, tj. dodatnog treniranja celog modela kako bi se bolje prilagodio konkretnom zadatku i poboljšao preciznost.

In [1]:
!pip -q install tensorflow

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Dataset.zip to Dataset.zip


In [4]:
import os
import zipfile

zip_path = 'Dataset.zip'
extract_path = 'dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [5]:
# Učitavanje slika iz foldera, automatski određujući klase prema podfolderima

import keras
import tensorflow as tf
import numpy as np

dataset = keras.utils.image_dataset_from_directory(
    'dataset/Dataset', # Putanja do dataset-a
    image_size = (224,224), # Dimenzije na koje će sve slike biti skalirane
    batch_size = 32 # Broj slika po batch-u
    )

print(f'Classes: {dataset.class_names}' )

number_of_classes = len(dataset.class_names) # Ukupan broj klasa, odnosno broj izlaza

# Postupak razdvajanja na slike(X) i labele(Y)

X = []
Y = []

for image,label in dataset:
  X.append(image)
  Y.append(label)

# Spajanje tensor-a, kako bi mogao da se vrši trening

X = tf.concat(X, axis = 0)
Y = tf.concat(Y, axis = 0)

Found 983 files belonging to 10 classes.
Classes: ['Alpine', 'Aston Martin', 'Ferrari', 'Haas', 'Kick Sauber', 'McLaren', 'Mercedes', 'Racing Bulls', 'Red Bull', 'Williams']


In [6]:
# Deljenje podataka na trening i test set

from sklearn.model_selection import train_test_split

X_np = X.numpy()
Y_np = Y.numpy()

X_train, X_test, Y_train, Y_test = train_test_split(X_np, Y_np, test_size = 0.2, random_state = 42)

In [7]:
# Kreiranje normalizacionog sloja koji vrši normalizacija na ulazne podatke, prema ResNet50 uputstvima

normalization_layer = keras.layers.Lambda(keras.applications.resnet50.preprocess_input)

# Kreiranje augmentacionog sloja, kako bi model bolje generalizovao i kako bi se povećao broj ulaznih podataka

data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.05),
    keras.layers.RandomZoom(0.05),
    keras.layers.RandomContrast(0.05),
    keras.layers.RandomBrightness(0.05)
])

In [8]:
# Učitavanje RestNet50 modela, ali bez poslednjeg sloja

base_model = keras.applications.ResNet50(weights = "imagenet", input_shape = (224,224,3), include_top = False)
base_model.trainable = False

# Kreiranje modela

model = keras.Sequential([
    data_augmentation,
    normalization_layer,
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(1024, activation="relu"),
    keras.layers.Dense(1024,activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(number_of_classes, activation = "softmax")
])

# Kompajliranje modela

model.compile(optimizer = keras.optimizers.Adam(learning_rate=1e-4), metrics = ["accuracy"], loss = keras.losses.SparseCategoricalCrossentropy())


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [9]:
# Prikaz arhitekture modela

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [10]:
# Definisanje callback funkcije u slučaju da model stagnira sa treniranjem

callbacks = [keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)]

# Treniranje modela

model.fit(X_train, Y_train, batch_size = 64, epochs = 20, validation_split=0.1, callbacks=callbacks)

Epoch 1/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 616ms/step - accuracy: 0.1293 - loss: 2.5201 - val_accuracy: 0.3165 - val_loss: 1.9902
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 299ms/step - accuracy: 0.3204 - loss: 1.9457 - val_accuracy: 0.4810 - val_loss: 1.6883
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 351ms/step - accuracy: 0.4557 - loss: 1.6212 - val_accuracy: 0.6962 - val_loss: 1.4397
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 298ms/step - accuracy: 0.5730 - loss: 1.4177 - val_accuracy: 0.6962 - val_loss: 1.2818
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 299ms/step - accuracy: 0.5985 - loss: 1.2291 - val_accuracy: 0.6962 - val_loss: 1.1745
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 277ms/step - accuracy: 0.6929 - loss: 1.0348 - val_accuracy: 0.7595 - val_loss: 0.9954
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 305ms/step - accuracy: 0.7314 - loss: 0.9102 - val_accuracy: 0.7595 - val_loss: 0.9340
Epoch 8/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 273ms/step - accuracy: 0.7489 - loss: 0.8529 - val_accuracy: 

Fine tuning

In [11]:
# Nakon osnovnog treniranja otključavaju se slojevi baznog modela i primenjuje se fine tuning

#for layer in base_model.layers[100:]:
  #layer.trainable = True

base_model.trainable = True

# Kompajliranje tog novog modela

model.compile(optimizer = keras.optimizers.Adam(1e-5), loss = keras.losses.SparseCategoricalCrossentropy(), metrics = ['accuracy'])

# Treniranje modela

model.fit(X_train,Y_train, batch_size = 64, epochs = 15, callbacks=callbacks, validation_split=0.1)


Epoch 1/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.7452 - loss: 0.7556 - val_accuracy: 0.6962 - val_loss: 0.7300
Epoch 2/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 998ms/step - accuracy: 0.8709 - loss: 0.4800 - val_accuracy: 0.6582 - val_loss: 0.8745
Epoch 3/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 998ms/step - accuracy: 0.9189 - loss: 0.3566 - val_accuracy: 0.7089 - val_loss: 0.7215
Epoch 4/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 988ms/step - accuracy: 0.9371 - loss: 0.2961 - val_accuracy: 0.7468 - val_loss: 0.5736
Epoch 5/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.9580 - loss: 0.2336 - val_accuracy: 0.8101 - val_loss: 0.4800
Epoch 6/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 999ms/step - accuracy: 0.9678 - loss: 0.1761 - val_accuracy: 0.8608 - val_loss: 0.4086
Epoch 7/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 975ms/step - accuracy: 0.9787 - loss: 0.1574 - val_accuracy: 0.8987 - val_loss: 0.3558
Epoch 8/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 997ms/step - accuracy: 0.9735 - loss: 0.1585 - val_accuracy: 

In [12]:
# Određivanje predikcija na test set-u

y_pred_probs = model.predict(X_test)

# Dobijene verovatnoće se prebacuju u stvarne klase

y_pred = np.argmax(y_pred_probs, axis=1)

print(y_pred)

7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 471ms/step
[6 5 4 6 4 6 6 3 4 7 5 4 2 0 9 6 9 7 5 0 5 3 5 3 7 0 4 4 0 5 5 3 0 4 9 0 5
 7 6 7 7 5 0 2 0 3 2 0 2 0 1 1 7 5 3 5 3 3 5 9 9 6 0 8 8 9 2 4 8 2 4 1 8 1
 6 1 5 1 5 2 8 6 7 4 1 1 3 7 7 3 4 3 8 2 6 6 6 9 3 1 2 8 1 2 5 1 9 6 7 2 3
 5 7 3 7 8 9 1 9 0 4 2 4 0 4 7 0 3 2 4 6 3 0 8 6 9 5 2 6 6 5 3 3 6 9 2 1 4
 4 9 3 6 0 4 1 2 8 4 2 5 0 7 8 0 4 8 4 1 2 7 3 6 0 9 9 9 6 0 9 6 5 1 9 7 9
 2 3 3 4 3 6 9 8 7 9 9 0]


In [13]:
# Računanje tačnosti

accurate = np.sum(y_pred == Y_test)
total = len(Y_test)

print(f'Accuracy: {accurate/total}')

Accuracy: 0.9238578680203046


In [14]:
# Prikazivanje statistike po klasama

from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred, target_names=dataset.class_names))

              precision    recall  f1-score   support

      Alpine       0.95      0.87      0.91        23
Aston Martin       1.00      0.89      0.94        18
     Ferrari       0.89      0.89      0.89        19
        Haas       0.78      1.00      0.88        18
 Kick Sauber       1.00      1.00      1.00        22
     McLaren       1.00      0.91      0.95        22
    Mercedes       0.87      0.91      0.89        22
Racing Bulls       1.00      0.95      0.97        19
    Red Bull       0.85      0.85      0.85        13
    Williams       0.91      0.95      0.93        21

    accuracy                           0.92       197
   macro avg       0.93      0.92      0.92       197
weighted avg       0.93      0.92      0.92       197

